In [135]:
from numba import jit,vectorize
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import numba
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st  #pour chi- deu test
import seaborn as sb
import statsmodels.api as sm # pour le modele logistique avec le summary
from statsmodels.formula.api import ols #pour anova one way test

%matplotlib inline

In [192]:
gro_dtypes = {
    'from': 'category',
    'to': 'category',
}
data=pd.read_csv("flights_train.csv.bz2")
data["flight_date"]=pd.to_datetime(data["flight_date"])
data.set_index("flight_date",inplace=True)
data=pd.get_dummies(data, drop_first=True)
data.head()


,avg_weeks,target,std_weeks,from_BOS,from_CLT,from_DEN,from_DFW,from_DTW,from_EWR,from_IAH,...,to_LAX,to_LGA,to_MCO,to_MIA,to_MSP,to_ORD,to_PHL,to_PHX,to_SEA,to_SFO
flight_date,,,,,,,,,,,,,,,,,,,,,
2012-06-19,12.875000,12.331296,9.812647,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-09-10,14.285714,10.775182,9.466734,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-10-05,10.863636,11.083177,9.035883,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2011-10-09,11.480000,11.169268,7.990202,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2012-02-21,11.450000,11.269364,9.517159,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [145]:
X,y=data.loc[:,(data.columns!="target")],data.loc[:,(data.columns=="target")]
X_train,X_test,y_train,y_test=train_test_split(data.loc[:,(data.columns!="target")],data.loc[:,data.columns=="target"],test_size=0.20, random_state=42,shuffle=True)

In [167]:
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from skopt.space.space import Real,Integer,Categorical
from skopt import BayesSearchCV
from sklearn.pipeline import Pipeline
ada_search = {
    'model': [AdaBoostRegressor()],
    'model__learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'model__n_estimators': Integer(1, 1000),
    'model__loss': Categorical(['linear', 'square', 'exponential'])
}
gb_search = {
    'model': [GradientBoostingRegressor()],
    'model__learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'model__n_estimators': Integer(1, 1000),
    'model__loss': Categorical(['ls', 'lad', 'quantile'])
}

In [ ]:
pipe = Pipeline([('model', GradientBoostingRegressor())])
opt = BayesSearchCV(
    pipe,
    [(ada_search, 100), (gb_search, 100)],
    cv=5
)
opt.fit(X_train,np.ravel(y_train))

In [ ]:
print(f"validation score: {opt.best_score_}")
print(f"test score: {opt.score(X_test,y_test)}")
print(f"best_params: {str(opt.best_params_)}")

In [187]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score,RepeatedKFold,RepeatedStratifiedKFold
regr = RandomForestRegressor()
regr.fit(X, np.ravel(y))
# evaluate the model
cv = RepeatedKFold(n_splits=5, random_state=1)
n_scores = cross_val_score(regr, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('RMSE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

RMSE: -0.792 (0.022)


In [170]:
regr = RandomForestRegressor(n_jobs=-1)
regr.fit(X_train, np.ravel(y_train))
y_pred=regr.predict(X_test)
print(f"the score of our model is : {regr.score(X_test,y_test)}")
from sklearn.metrics import mean_squared_error
print(f"mean_squared_error is : {mean_squared_error(y_pred,y_test,squared=False)}")

the score of our model is : 0.2979616808810097
mean_squared_error is : 0.7951473932430938


In [183]:
opt = BayesSearchCV(
    RandomForestRegressor(),
   {    'n_estimators'=Real(0.005, 0.9, prior="log-uniform"),
        'min_samples_split': Integer(2,100),
        'min_samples_leaf': Integer(2,100),
 },
    cv=3, 
    n_jobs=-1,
    n_iter=32,
    random_state=0
)
opt.fit(X_train,np.ravel(y_train))

C:\Users\zechc\.conda\envs\datacamp\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=32, n_jobs=-1,
              random_state=0,
              search_spaces={'min_samples_leaf': Integer(low=2, high=100, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=100, prior='uniform', transform='normalize')})

In [184]:
y_pred=opt.predict(X_test)
print(f"the score of our model is : {opt.score(X_test,y_test)}")
from sklearn.metrics import mean_squared_error
print(f"mean_squared_error is : {mean_squared_error(y_pred,y_test,squared=False)}")

the score of our model is : 0.38812460668305404
mean_squared_error is : 0.7423329273838701


In [185]:
print(f"validation score: {opt.best_score_}")
print(f"test score: {opt.score(X_test,y_test)}")
print(f"best_params: {str(opt.best_params_)}")

validation score: 0.35631055728676336
test score: 0.38812460668305404
best_params: OrderedDict([('min_samples_leaf', 2), ('min_samples_split', 37)])


RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1)

RepeatedKFold(n_repeats=10, n_splits=5, random_state=1)